In [1]:
import torch

from lstm import LSTM

In [2]:
x = torch.rand(4, 27, 128)
m = LSTM(128, 64)

In [3]:
a, (b, c) = m(x)

In [4]:
a.shape, b.shape, c.shape

(torch.Size([4, 27, 64]), torch.Size([4, 64]), torch.Size([4, 64]))